In [4]:
import math
import numpy as np
from PyCO2SYS import CO2SYS
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [7]:
## SHIAMI SPECIATION MODEL [SCRIPT VERSION]
# SHIAMI calculates speciation for a number of elements in seawater given sufficient input. 
# CORRESPONDENCE: michelle.madin@hotmail.co.uk 
# ALSO: available as a function ... see SHI_AMI.m

## Declare global variables
##  Assign values to global variables
mjCaMax = 5
mnCaMax = 37
CaMax = mjCaMax + mnCaMax

mjAnMax = 7
mnAnMax = 19
AnMax = mjAnMax + mnAnMax

NeutMax = 7

## Intiating Speciation calculation:
# In order to initialise the calculation of speciation in an aqueous solution, one needs 
# the following:
## --------------------------------------------------------------------------------------------------------------
## 1) Controlling variables:
# NB:- Like Hain et al, SHIAMI uses modern seawater relationships among 
# Practical Salinity Scale salinity (UNESCO, 1981), ionic strength (I), and chlorinity(Cl) 
# (Dickson, 2010) ---> {Denoted by *}

S  = 35                         #        Salinity*: 0-40 psu, Practical Salinity Scale salinity (UNESCO, 1981)
I  = 19.924*S/(1000-1.005*S)    #        Ionic strength*: Determined from Salinity
Tc = 25                         #        Temperature: 0-50 degrees C
P  = 1                          #        Pressure: Via CO2SYS, Information on pressure dependence is limited
pH = 8.1                        #        pH: seawater scale 

## --------------------------------------------------------------------------------------------------------------
## 2) Composition of background electrolyte i.e standard seawater [Millero et al., 2008]
#
# The code assumes a simplified seawater composition  (mol of solute/kg of seawater) 
# composed of the following major ions relative to total salinity: 
#
# Na+, K+, Mg2+, Ca2+, Sr2+
# Cl-, SO42-, CO32- HCO3-  Br-, F-, B(OH)4 
#
# In the open ocean, the principal chemical constituents in seawater have nearly identical 
# concentrations relative to one another.  Consequently, by determining the concentration 
# of one major chemical constituent of seawater, you can determine the concentrations of 
# each individual principal chemical element present in seawater.
#
# The constituent originally selected to characterize a given sample of seawater or 
# other natural waters is chlorinity, Cl (). 
#
# SHIAMI uses the following relationship between chlorinity* and salinity:
#                  S() = 1.80655 x Chlorinity () 
#                  Cl(mol/kg) = 0.5458696 * S/35
# ****************************************************************************************************************************
# FOR EXAMPLE:
# [Na] = 0.5564924 {g of solute /Kg of solvent} / Cl() .... (1)                
# 0.5564924 * 1/(22.98976928)  mol/g  = 0.024211 .... (2)         
# 0.024211 * Cl  = 0.024211 * S/1.80655 = (0.0134*S) mol/kg  
# Using S = 35.000: 
# [Na] =  0.46897 mol/kg of solvent ... (3)                                               
# 0.46897 * (S/1.80655)/(35/1.80655)  = 0.46897 * (S/35)
#
# (1) Table 4, column 5; Millero, 2008
# (2) Inverse of Table 4, column 6; Millero, 2008
# (3) Table 4, column 7; Millero, 2008
# *************************************************************************************************************
## --------------------------------------------------------------------------------------------------------------    
## 2a) Cations 

mT_cat = [0, 0, 0, 0, 0]

mT_cat[0] = 0.4689674*S/35         # [Na]           
mT_cat[1] = 0.0102077*S/35         # [K]
mT_cat[2] = 0.0528171*S/35         # [Mg]
mT_cat[3] = 0.0102821*S/35         # [Ca]      
mT_cat[4] = 0.0000907*S/35         # [Sr]
## --------------------------------------------------------------------------------------------------------------
## 2b) Carbonate system
#
# MIAMI and MyAMI relates total concentration of HCO3- and CO32- to salinity.  The use of 
# salinity limits the models use for waters of different compositions (Turner, et al 2016). 
# Several software packages are available for C02 systems that, if incorporated, could 
# significant speciation models.  I have made use of the MATLAB version of CO2SYS 
# (Lewis et al. 1998; Van Heuven et al. 2011). Provided with sufficient input, CO2SYS 
# returns the state of the carbonate system  of the water sample. Specifically, SHIAMI
# uses the CO2SYS model to output the total concentration of HCO3 and CO3 (umol/kgSW). 
#
# To characterize the various components of the carbonate system using CO2SYS one must 
# measure, at least two of the following:
#
#  1)	pH
#  2)	Alkalinity, (TA)
#  3)	Total CO2, (TCO2)
#  4)	Partial pressure of CO2, (pCO2)
#
# And both: 
#
# 1) Concentration of silicate  
# 2) Concentration of phosphate 
#
# Here we use CO2SYS ( MATLAB-version) to returns the state of the carbonate system of 
# our water sample: 
#
# Addional info: alk=2400, si=50, po4=2, dissociation constants: 'Millero 2010'
Xpar1type = 1           # The first parameter supplied is of type "1", which is "alkalinity"
Xpar1     = 2400        # value of the first parameter
Xpar2type = 3           # The first parameter supplied is of type "3", which is "pH"
Xpar2     = pH          # value of the second parameter
Xsal      = S           # Salinity of the sample
Xtempin   = Tc          # Temperature at input conditions
Xpresin   = 10.1325     # Pressure    at input conditions
Xtempout  = np.nan      # Temperature at output conditions - doesn't matter here
Xpresout  = np.nan      # Pressure    at output conditions - doesn't matter here
Xsil      = 50          # Concentration of silicate  in the sample (in umol/kg)
Xpo4      = 2           # Concentration of phosphate in the sample (in umol/kg)
XpHscale  = 2           # pH scale at which the input pH is reported ("1" means "Seawater Scale")
Xk1k2c    = 14          # Choice of H2CO3 and HCO3- dissociation constants K1 and K2 ("14" means "Millero 2010")
Xkso4c    = 1           # Choice of HSO4- dissociation constants KSO4 ("1" means "Dickson")

A = CO2SYS(Xpar1, Xpar2, Xpar1type, Xpar2type, Xsal, Xtempin, Xtempout, Xpresin, Xpresout, Xsil, Xpo4, XpHscale, Xk1k2c, Xkso4c)
# FOR MORE INFO: SEE CO2SYS.m
## --------------------------------------------------------------------------------------------------------------
##  2c) Anions

mT_an = [0, 0, 0, 0, 0, 0, 0]

mT_an[0]  = 0.5458696*S/35          # [Cl]
mT_an[1]  = 0.0282352*S/35          # [SO4]
mT_an[2]  = 0.00024694*S/35         # [CO3]  : A[6] * 1e-6   :   0.0002390*S/35  :    0.000246937727254629       
mT_an[3]  = 0.00179143*S/35         # [HCO3] : A[5] * 1e-6   :   0.0017177*S/35  :    0.001791429070585
mT_an[4]  = 0.0008421*S/35          # [Br]
mT_an[5]  = 0.0000683*S/35          # [F]
mT_an[6]  = 0.0001008*S/35          # [B(OH)4] 
OH = 10**(-14+8.1)                  # A[9] * 1e-6 
## --------------------------------------------------------------------------------------------
## 3) Free Activity coefficients [gammaV3.m; gammaNV3.m]
# Like Millero Model, we calculate gamma using Pitzer equations for :
#
# Major Cations                  Minor Cations 
# Na, K, Mg, Ca, Sr              H, Li, Rb, Cs, NH4, Ba, Mn, Fe(II), Co, Ni, Cu(II), Zn, UO2,
#                                Be, Cd, Pb(II), Cu(I)
#                               
#                                REE
#                                La, Ce, Pr, Nd, Pm, Sm, Eu, Gd, Tb, Dy, Ho, Er, Tm, Yb, Lu, Y
#                                
#                                Trivalent
#                                Al, Ga, In, Fe(III)
#
#
# Major Anions                   Minor Anion
# Cl, SO4, CO3, HCO3             OH, HSO4, HS,  I, ClO3, ClO4, BrO3, CNS, NO2, NO3, H2PO4, HPO4,
# Br, F, B(OH)4                  PO4, H2AsO4, HAsO4, AsO4, HSO3, SO3, Acetate
#
# Neutral:
# NH3, B(OH)3, H3PO4, H2S, SO2, HF, CO2
#
# Charged ion pairs whose pitzer parameters are available
# MgOH*  MgF*  CaF*
#
#
# Neutral ion pairs and charged ion pairs for which no pitzer data is available: 
# For a number of ion pairs, we have assigned assumed activity coefficients.
# For detailed description of assumptions made, see "gammaIP.m" or type "help gammaIP" 
# in the command window .

# Pitzer parameters that are needed  are generated by (and can be edited in) CreatePitzerV3.m
## ****************************************************************************************************************************
gF_cat, gF_an, gIP1, gdivIP, gtrivIP = gammaFV3(Tc, I, mT_cat,mT_an)
gN = gammaNV3(Tc,I,mT_cat,mT_an)
# gmjIP, mnCatIP, gdivIP, gtrivIP = gammaIP(Tc,I, mT_cat, mT_an)
## ****************************************************************************************************************************
## --------------------------------------------------------------------------------------------------------------
## 4) Thermodynamic/conditional association constants
# Equilbrium constants used in the chemical equilibria can be:
#
# Thermodynamic constants K:                - expressed in terms of activity 
#                                           - Functions of temperature and pressure only. 
#
# Stoichiometric/conditional constants K?:  - expressed in terms of concentrations 
#                                           - functions of temperature, pressure and 
#                                             solution composition. 
#
# "This dependence on solution compositionvia the activity coefficients limits the 
# practical value  of stoichiometric constants.  Values of the stoichiometric constant 
# K* measured in  normal seawater can be used only for solutions of seawater composition 
# and for the salinity, or salinities, for which K* has been determined.  Any variations 
# from seawater compositionsuch as might be found in enclosed seas ,pore waters, 
# and some estuarieswill lead to changes  in the value of ... activity coefficients 
# in the  equation ... and therefore a change in K*" (TURNER ET AL 2016)
#
# In SHIAMI: 
# 1) Thermodynamic log Ks are compiled from various sources. SEE "Kthermo.m" and 
#    "thermodatabase1.m"
# 
# 2) Desired stoichiometric constants are calculated from thermodynamic log Ks using the 
#    model calculated free activity coefficients.
# 
# ****************************************************************************************************************************
# [mjKthermo, mnCat_Kthermo, divKthermo, trivKthermo, mnAn_Kthermo] = Kthermo(Tc)
# [mjKcond, mnCat_Kcond, divKcond, trivKcond]  = Kcond(Tc, I, mT_cat,mT_an);
# ****************************************************************************************************************************
## ###########################################################################################################
## CALCULATING SPECIATION: 
# Using the parameters listed above, one can determine the distribution of elements 
# in an aqueous solution  via the following means:
## --------------------------------------------------------------------------------------------------------------
## 1) Calculating "alpha" or the fraction of free ions in a solution of fixed composition [alphaV3.m] 
# To calculate the percentage/fraction of free cation or anion in the solution, we
# use the "ion association/ion-pairing model. We solve a series of linear
# equations through an iterative method which is initiated by setting the
# alpha_cat = 1 i.e [M]F = [M]T. 

# ****************************************************************************************************************************
alpha_cat, alpha_an, mF_an, mF_cat, itera = alphasV3(Tc, I, mT_cat,mT_an,OH)
# ****************************************************************************************************************************

## --------------------------------------------------------------------------------------------------------------
## 2) Calculating speciation
# Using alpha determined above, it is possible to determine the speciation:
# ****************************************************************************************************************************
mjcat_spec, mjcat_FREE, mjan_spec, mjan_FREE, mncat_spec, mncat_FREE, div_spec, div_FREE, triv_spec, triv_FREE = speciation (Tc, I, mT_cat,mT_an, OH)
# ****************************************************************************************************************************

## ###########################################################################################################
## Correcting activity coefficients for complex formation: 
# From Hain et al 2015:
# 'There are two reasons why the activity of a given chemical species (e.g., CO32-) is different from its
# stoichiometric concentration: 

# (1) The electrostatic ion-ion interactions in electrolyte solutions typically reduce the activity 
#     of a given species below its free concentration Debye and Hückel, 1923; Pitzer, 1991). 
#     [This is accounted for by free activity coeefficients]

# (2) Some ions form strong bonds with each other (i.e., complex formation or ion pairing;
#     Garrels and Thompson, 1962) such that only a fraction of that species can be said to be free'
#     [This is accounted for by the desired total activity coeefficients.]

## Calculating Total Activity coefficents [gammaTV3.m]
# ai = mf * yf               (1)
# mf = mT * alpha            (2)
# ai = mT * alpha * yF       (3) = (2) -> (1)
# yT = alpha * yF            (4)
# ai = mT * yT               (5) = (4) -> (3)

# When the ion does not form an ion pair or alpha = 1.0 the values of yT = yF. 
# If the value of yF is determined for a solution (e.g. Na-Mg-C1-SO4) using the
# Pitzer equations, the ion pairing model can be used to account for strong
# interactions between i (Cu 2+) and minor anions (CO~-):

# Using the alpha determined via iteration,  we can convert yF to yT: 

# ****************************************************************************************************************************
gT_cat, gT_an = gammaTV3 (alpha_an, alpha_cat, gF_an,gF_cat)
# ****************************************************************************************************************************

NameError: name 'gammaFV3' is not defined